In [13]:
import numpy as np
import pandas
import scipy
from sklearn import svm
import matplotlib.pyplot as plt



In [14]:
english = pandas.read_csv("train_cap2018.csv")

In [15]:
target = english['level1']
target.head()
print(target.shape)

data = english.loc[:, "sentences":"punct"]
print(data.shape)

(27310,)
(27310, 5)


In [16]:
fig = plt.figure()
y = english.groupby('level1').sentences.count()
plt.bar(y.keys(), y.values)
plt.show()

In [17]:
# clf = svm.SVC(gamma=0.001, C=100.)
from sklearn.svm import LinearSVC

clf = LinearSVC(random_state=0) 
data_cleaned = data.T.dropna().T

clf.fit(data_cleaned, target)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)

In [12]:
data_cleaned.head()

print(data_cleaned[:1])
clf.predict(data_cleaned[:1])

   sentences  words  letters.all  syllables  punct  avg.sentc.length  \
0       16.0  233.0       1116.0      363.0   32.0           14.5625   

   avg.word.length  avg.syll.word  sntc.per.word       TTR    ...     \
0           4.7897        1.55794        0.06867  0.587983    ...      

   HD-D (vocd-D)  Herdan's C  Maas a  Maas lgV0   MTLD  Root TTR  Summer  \
0          34.65         0.9     0.2       4.96  70.74      8.98    0.88   

   TTR.1  Uber index  Yule's K  
0   0.59        24.3    146.62  

[1 rows x 56 columns]


array(['B2'], dtype=object)

In [14]:
from sklearn.model_selection import cross_val_score
%time listerr = cross_val_score(clf, data_cleaned, target, cv=5)
print listerr

CPU times: user 2min 18s, sys: 100 ms, total: 2min 18s
Wall time: 2min 18s
[0.59542543 0.61266703 0.39454412 0.45421245 0.42509158]


In [19]:
from sklearn.preprocessing import StandardScaler
data_standard = StandardScaler().fit_transform(data_cleaned)
corr = np.corrcoef(data_standard.T)
corr.shape

(5, 5)

In [20]:
import seaborn as sea
sea.heatmap(corr)
corr

array([[1.        , 0.66837766, 0.64175596, 0.64342692, 0.73276522],
       [0.66837766, 1.        , 0.98341281, 0.98538917, 0.71997432],
       [0.64175596, 0.98341281, 1.        , 0.99595057, 0.70399435],
       [0.64342692, 0.98538917, 0.99595057, 1.        , 0.71004385],
       [0.73276522, 0.71997432, 0.70399435, 0.71004385, 1.        ]])